In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from activation_helper import sigmoid
from make_dataset_helper import prepare_dataset
from sklearn.model_selection import train_test_split

    
    return X_train, X_test, y_train, y_test


def main():
    X_train, X_test, y_train, y_test = prepare_dataset()


if __name__ == '__main__':
    main()